### Importing packages for original notebook

In [1]:
# Original metrics imports

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

# set dimemsion names for xarray datasets
dim_name_level  = 'lev'
dim_name_sample = 'sample'

### Importing data_utils.py

In [2]:
from data_utils import *

2023-08-15 23:10:03.117465: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-15 23:10:04.092779: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Sungduk version

In [3]:
FN_MODEL_OUTPUT = {'MLP':  '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/001_backup_phase-7_retrained_models_step2_lot-147_trial_0027.best.h5.npy',
                   'RPN':  '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/rpn_pred_v1_stride6.npy',
                   'CNN':  '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/val_predict_cnn_reshaped_stride6_FINAL.npy',
                   # 'cVAE': './model_outputs/cvae_preds_bestcrps.h5',
                   'cVAE': '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/cvae.h5',
                   'HSR': '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/hsr_preds_bestcrps.h5',
                  }

# model name
# (model name is used for the output)
model_name = 'MLP'

# input of validation dataset (npy)
fn_x_true = '/ocean/projects/atm200007p/jlin96/neurips_proj/e3sm_train_npy/val_input_stride6.npy'

# true output of validation dataset (npy)
fn_y_true = '/ocean/projects/atm200007p/jlin96/neurips_proj/e3sm_train_npy/val_target_stride6.npy'

# Model predicted output of varlidation dataset (npy)
fn_y_pred = FN_MODEL_OUTPUT[model_name]

# model grid information (nc)
fn_grid = '/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/grid_info/E3SM-MMF_ne4_grid-info.orig.nc'

# normalization scale factors (nc)
fn_mli_mean  = '/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/norm_factors/mli_mean.nc'
fn_mli_min   = '/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/norm_factors/mli_min.nc'
fn_mli_max   = '/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/norm_factors/mli_max.nc'
fn_mlo_scale = '/ocean/projects/atm200007p/jlin96/neurips_proj/ClimSim_release/norm_factors/mlo_scale.nc'

# fn_save_output
fn_save_metrics = f'./metrics/{model_name}.metrics.csv'
fn_save_metrics_avg = f'./metrics/{model_name}.metrics.lev-avg.csv'

# physical constatns from (E3SM_ROOT/share/util/shr_const_mod.F90)
grav    = 9.80616    # acceleration of gravity ~ m/s^2
cp      = 1.00464e3  # specific heat of dry air   ~ J/kg/K
lv      = 2.501e6    # latent heat of evaporation ~ J/kg
lf      = 3.337e5    # latent heat of fusion      ~ J/kg
ls      = lv + lf    # latent heat of sublimation ~ J/kg
rho_air = 101325./ (6.02214e26*1.38065e-23/28.966) / 273.15 # density of dry air at STP  ~ kg/m^3
                                                            # ~ 1.2923182846924677
                                                            # SHR_CONST_PSTD/(SHR_CONST_RDAIR*SHR_CONST_TKFRZ)
                                                            # SHR_CONST_RDAIR   = SHR_CONST_RGAS/SHR_CONST_MWDAIR
                                                            # SHR_CONST_RGAS    = SHR_CONST_AVOGAD*SHR_CONST_BOLTZ
rho_h20 = 1.e3       # density of fresh water     ~ kg/m^ 3

vars_mlo_energy_conv = {'ptend_t':cp,
                        'ptend_q0001':lv,
                        'cam_out_NETSW':1.,
                        'cam_out_FLWDS':1.,
                        'cam_out_PRECSC':lv*rho_h20,
                        'cam_out_PRECC':lv*rho_h20,
                        'cam_out_SOLS':1.,
                        'cam_out_SOLL':1.,
                        'cam_out_SOLSD':1.,
                        'cam_out_SOLLD':1.
                       }

# load input dataset
x_true = np.load(fn_x_true).astype(np.float64)
y_true = np.load(fn_y_true).astype(np.float64)
if fn_y_pred[-3:] == '.h5':
    y_pred = xr.open_dataset(fn_y_pred)['pred'].values
else:
    y_pred = np.load(fn_y_pred).astype(np.float64)
N_samples = y_pred.shape[0] 

# load norm/scale factors
mlo_scale = xr.open_dataset(fn_mlo_scale)
mli_mean  = xr.open_dataset(fn_mli_mean)
mli_min   = xr.open_dataset(fn_mli_min)
mli_max   = xr.open_dataset(fn_mli_max)

# load grid information
ds_grid = xr.open_dataset(fn_grid) # has ncol:384
N_ncol = len(ds_grid['ncol']) # length of ncol dimension (nlat * nlon)

# make area-weights
ds_grid['area_wgt'] = ds_grid['area'] / ds_grid['area'].mean('ncol')

# map ds_grid's ncol dimension -> the N_samples dimension of npy-loayd arrays (e.g., y_pred)
to_xarray = {'area_wgt': (dim_name_sample,np.tile(ds_grid['area_wgt'], int(N_samples/len(ds_grid['ncol'])))),
            }
to_xarray = xr.Dataset(to_xarray)

# add nsample-mapped grid variables back to ds_grid
ds_grid = xr.merge([ds_grid  [['P0', 'hyai', 'hyam','hybi','hybm','lat','lon','area']],
                    to_xarray[['area_wgt']]])

# list of ML output variables
vars_mlo = ['ptend_t','ptend_q0001','cam_out_NETSW','cam_out_FLWDS','cam_out_PRECSC',
            'cam_out_PRECC','cam_out_SOLS','cam_out_SOLL','cam_out_SOLSD','cam_out_SOLLD'] # mlo mean ML output.

# length of each variable
vars_mlo_len = {'ptend_t':60,
                'ptend_q0001':60,
                'cam_out_NETSW':1,
                'cam_out_FLWDS':1,
                'cam_out_PRECSC':1,
                'cam_out_PRECC':1,
                'cam_out_SOLS':1,
                'cam_out_SOLL':1,
                'cam_out_SOLSD':1,
                'cam_out_SOLLD':1
               }

# map the length of dimension to the name of dimension
len_to_dim = {60:dim_name_level,
              N_samples: dim_name_sample}

# Here, we first construct a dictionary of {var name: (dimension name, array-like)},
# then, map the dictionary to an Xarray Dataset.
# (ref: https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html)

DS = {}

for kds in ['true', 'pred']:
    if kds=='true':
        work = y_true
    elif kds=='pred':
        work = y_pred

    # [1] Construct dictionary for xarray dataset
    #     format is key for variable name /
    #               value for a turple of (dimension names, data).
    to_xarray = {}
    for k, kvar in enumerate(vars_mlo):

        # length of variable (ie, number of levels)
        kvar_len = vars_mlo_len[kvar]

        # set dimensions of variable
        if kvar_len == 60:
            kvar_dims = (dim_name_sample, dim_name_level)
        elif kvar_len == 1:
            kvar_dims = dim_name_sample

        # set start and end indices of variable in the loaded numpy array
        # then, add 'kvar':(kvar_dims, <np_array>) to dictionary
        if k==0: ind1=0
        ind2 = ind1 + kvar_len

        # scaled output
        kvar_data = np.squeeze(work[:,ind1:ind2])
        # unscaled output
        kvar_data = kvar_data / mlo_scale[kvar].values

        to_xarray[kvar] = (kvar_dims, kvar_data)

        ind1 = ind2

    # [2] convert dict to xarray dataset
    DS[kds] = xr.Dataset(to_xarray)

    # [3] add surface pressure ('state_ps') from ml input
    # normalized ps
    state_ps =  xr.DataArray(x_true[:,120], dims=('sample'), name='state_ps')
    # denormalized ps
    state_ps = state_ps * (mli_max['state_ps'] - mli_min['state_ps']) + mli_mean['state_ps']
    DS[kds]['state_ps'] = state_ps

    # [4] add grid information
    DS[kds] = xr.merge([DS[kds], ds_grid])

    # [5] add pressure thickness of each level, dp
    # FYI, in a hybrid sigma vertical coordinate system, pressure at level z is
    # P[x,z] = hyam[z]*P0 + hybm[z]*PS[x,z],
    # where, hyam and hybm are 
    tmp = DS[kds]['P0']*DS[kds]['hyai'] + DS[kds]['state_ps']*DS[kds]['hybi']
    tmp = tmp.isel(ilev=slice(1,61)).values - tmp.isel(ilev=slice(0,60)).values
    tmp = tmp.transpose()
    DS[kds]['dp'] = xr.DataArray(tmp, dims=('sample', 'lev'))

    # [6] break (sample) to (ncol,time)
    N_timestep = int(N_samples/N_ncol)
    dim_ncol     = np.arange(N_ncol)
    dim_timestep = np.arange(N_timestep)
    new_ind = pd.MultiIndex.from_product([dim_timestep, dim_ncol],
                                         names=['time', 'ncol'])
    DS[kds] = DS[kds].assign_coords(sample=new_ind).unstack('sample')

# [1] Weight vertical levels by dp/g that is equivalent to a mass of air within a grid cell per unit area [kg/m2]
# [2] Weight horizontal area of each grid cell by a[x]/mean(a[x]).
# [3] Unit conversion to a common energy unit

DS_ENERGY = {}
for kds in ['true','pred']:
    # Make a copy to keep original dataset
    DS_ENERGY[kds] = DS[kds].copy(deep=True)

    # vertical weighting / area weighting / unit conversion
    for kvar in vars_mlo:

        # [1] weight vertical levels by dp/g
        #     ONLY for vertically-resolved variables, e.g., ptend_{t,q0001}
        # dp/g = - \rho * dz
        if vars_mlo_len[kvar] == 60:
            DS_ENERGY[kds][kvar] = DS_ENERGY[kds][kvar] * DS_ENERGY[kds]['dp']/grav

        # [2] weight area
        #     for ALL variables
        DS_ENERGY[kds][kvar] = DS_ENERGY[kds]['area_wgt'] * DS_ENERGY[kds][kvar]

        # [3] convert units to W/m2
        #     for variables with different units, e.g., ptend_{t,q0001}, precsc, precc
        DS_ENERGY[kds][kvar] =  vars_mlo_energy_conv[kvar] * DS_ENERGY[kds][kvar]


In [4]:
ptend_t_check = DS_ENERGY['true']['ptend_t'][42,:,:].values
ptend_t_check

array([[ 1.50306918e-02,  2.49113763e-02,  6.33468952e-02, ...,
         4.47187562e+00,  1.90149031e+00, -1.43307518e+00],
       [ 1.77264882e-02,  2.30794595e-02,  7.46828344e-02, ...,
         4.25452349e+00,  1.38797065e+00, -2.30128018e+00],
       [ 1.49848568e-02,  3.32651641e-02,  6.22291081e-02, ...,
        -1.42263246e-01, -2.10414987e+00, -2.96664012e+00],
       ...,
       [-3.58889074e-03,  4.15972963e-03, -3.79946533e-02, ...,
        -5.83624483e+00, -4.72771262e+00, -1.04185159e+01],
       [-1.33169705e-03, -6.56815159e-04, -4.07090036e-02, ...,
        -3.12313991e+00, -6.76441827e+00, -1.30581222e+01],
       [-5.01235620e-03, -5.59752634e-03, -3.49614343e-02, ...,
        -1.15787190e+00, -7.14604993e+00, -1.59112957e+01]])

In [5]:
ptend_q_check = DS_ENERGY['true']['ptend_q0001'][49,:,:].values
ptend_q_check

array([[  0.        ,   0.        ,   0.        , ...,  -2.74203415,
          0.66898269,   1.57286221],
       [  0.        ,   0.        ,   0.        , ...,   0.74758948,
          4.52463957,   8.91198655],
       [  0.        ,   0.        ,   0.        , ..., -12.97049343,
        -11.32664595,   1.57625109],
       ...,
       [  0.        ,   0.        ,   0.        , ...,  -1.29968077,
          7.98803317,  10.84303509],
       [  0.        ,   0.        ,   0.        , ...,   2.84016395,
          8.48974947,  10.26747987],
       [  0.        ,   0.        ,   0.        , ...,  -4.6976471 ,
          0.13164726,   2.33216457]])

In [6]:
sols_check = DS_ENERGY['true']['cam_out_SOLS'][49,:].values
sols_check

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

### Jerry version

In [7]:
data_path = '/ocean/projects/atm200007p/walrus/for_jerry/train/'
norm_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/mooers_metrics/norm_factors/'
grid_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/mooers_metrics/test_data/E3SM-MMF_ne4_grid-info.orig.nc'
target_scoring_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/val_target_stride6.npy'
cvae_pred_path = '/ocean/projects/atm200007p/shared/neurips_proj/final_metrics/predictions/cVAE/cvae_preds_manual.h5'
hsr_pred_path = '/ocean/projects/atm200007p/shared/neurips_proj/final_metrics/predictions/HSR/hsr_preds_bestcrps.h5'
rpn_pred_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/rpn_pred_v1_stride6.npy'
cnn_pred_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/val_predict_cnn_reshaped_stride6_FINAL.npy'
mlp_pred_path = '/ocean/projects/atm200007p/jlin96/neurips_proj/figure_ingredients/001_backup_phase-7_retrained_models_step2_lot-147_trial_0027.best.h5.npy'

data_path = data_path
input_vars = ['state_t','state_q0001','state_ps','pbuf_SOLIN', 'pbuf_LHFLX', 'pbuf_SHFLX']
target_vars = ['ptend_t','ptend_q0001','cam_out_NETSW','cam_out_FLWDS','cam_out_PRECSC','cam_out_PRECC','cam_out_SOLS','cam_out_SOLL','cam_out_SOLSD','cam_out_SOLLD']
grid_info = xr.open_dataset(grid_path)
inp_mean = xr.open_dataset(norm_path + 'mli_mean.nc')
inp_max = xr.open_dataset(norm_path + 'mli_max.nc')
inp_min = xr.open_dataset(norm_path + 'mli_min.nc')
out_scale = xr.open_dataset(norm_path + 'mlo_scale.nc')

In [8]:
scoring_data = data_utils(data_path = data_path, 
                          input_vars = input_vars, 
                          target_vars = target_vars, 
                          grid_info = grid_info, 
                          inp_mean = inp_mean, 
                          inp_max = inp_max, 
                          inp_min = inp_min, 
                          out_scale = out_scale)

In [9]:
# set regular expressions for selecting data
scoring_data.set_regexps(data_split = 'scoring', regexps = ['E3SM-MMF.mli.0008-0[23456789]-*-*.nc', 
                                                            'E3SM-MMF.mli.0008-1[012]-*-*.nc', 
                                                            'E3SM-MMF.mli.0009-01-*-*.nc'])
# set temporal subsampling
scoring_data.set_stride_sample(data_split = 'scoring', stride_sample = 6)
# create list of files to extract data from
scoring_data.set_filelist(data_split = 'scoring')
# load target output
scoring_data.target_scoring = np.load(target_scoring_path)

In [10]:
scoring_data.model_names = ['cVAE','HSR','RPN','CNN', 'MLP']
preds = [scoring_data.get_pred_h5(load_path = cvae_pred_path), 
         scoring_data.get_pred_h5(load_path = hsr_pred_path), 
         scoring_data.get_pred_npy(load_path = rpn_pred_path), 
         scoring_data.get_pred_npy(load_path = cnn_pred_path), 
         scoring_data.get_pred_npy(load_path = mlp_pred_path)]
scoring_data.preds_scoring = dict(zip(scoring_data.model_names, preds))

In [11]:
assert np.sum(scoring_data.target_scoring-y_true) == 0
assert scoring_data.target_scoring.shape == y_true.shape

In [12]:
heating, moistening, netsw, flwds, precsc, precc, sols, soll, solsd, solld = \
    scoring_data.output_weighting(x_true, y_true)

In [13]:
heating.shape

(4380, 384, 60)

In [14]:
moistening.shape

(4380, 384, 60)

In [15]:
netsw.shape

(4380, 384)

In [16]:
flwds.shape

(4380, 384)

In [17]:
precsc.shape

(4380, 384)

In [18]:
precc.shape

(4380, 384)

In [19]:
sols.shape

(4380, 384)

In [20]:
soll.shape

(4380, 384)

In [21]:
solsd.shape

(4380, 384)

In [22]:
solld.shape

(4380, 384)

### Time to compare

In [39]:
assert(np.sum(DS_ENERGY['true']['ptend_t'].values - heating) == 0)
assert(DS_ENERGY['true']['ptend_t'].values.shape == heating.shape)

assert(np.sum(DS_ENERGY['true']['ptend_q0001'].values - moistening) == 0)
assert(DS_ENERGY['true']['ptend_q0001'].values.shape == moistening.shape)

assert(np.sum(DS_ENERGY['true']['cam_out_NETSW'].values - netsw) == 0)
assert(DS_ENERGY['true']['cam_out_NETSW'].values.shape == netsw.shape)

assert(np.sum(DS_ENERGY['true']['cam_out_FLWDS'].values - flwds) == 0)
assert(DS_ENERGY['true']['cam_out_FLWDS'].values.shape == flwds.shape)

assert(np.sum(DS_ENERGY['true']['cam_out_PRECSC'].values - precsc) == 0)
assert(DS_ENERGY['true']['cam_out_PRECSC'].values.shape == precsc.shape)

assert(np.sum(DS_ENERGY['true']['cam_out_PRECC'].values - precc) == 0)
assert(DS_ENERGY['true']['cam_out_PRECC'].values.shape == precc.shape)

assert(np.sum(DS_ENERGY['true']['cam_out_SOLS'].values - sols) == 0)
assert(DS_ENERGY['true']['cam_out_SOLS'].values.shape == sols.shape)

assert(np.sum(DS_ENERGY['true']['cam_out_SOLL'].values - soll) == 0)
assert(DS_ENERGY['true']['cam_out_SOLL'].values.shape == soll.shape)

assert(np.sum(DS_ENERGY['true']['cam_out_SOLSD'].values - solsd) == 0)
assert(DS_ENERGY['true']['cam_out_SOLSD'].values.shape == solsd.shape)

assert(np.sum(DS_ENERGY['true']['cam_out_SOLLD'].values - solld) == 0)
assert(DS_ENERGY['true']['cam_out_SOLLD'].values.shape == solld.shape)